# Setup N Virtual Machines for Multiple Users

Use Virtual Machine Scale Set (VMSS) to create *N* Virtual Machines (VMs), then invoke a script to setup multiple users on each VM instance. 

## Azure CLI Setup

In [5]:
AZURE_SUBSCRIPTION_ID = "1b365fe2-5882-4935-bd81-8027e0816b45"
LOCATION = "australiaeast"
VM_SIZE = "Standard_NC6s_v3"
RG_NAME = "mladsgpuaustraliaeast-test"
VMSS_NAME = "{}-vmss".format(RG_NAME)

# Number of VM instances in a scale set 
NUM_VM = 4

In [ ]:
!az login

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

In [2]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "1b365fe2-5882-4935-bd81-8027e0816b45",
  "isDefault": true,
  "name": "AICAT-VB-HAR",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "jumin@microsoft.com",
    "type": "user"
  }
}


## Create VMSS

In [6]:
!az group create --name {RG_NAME} --location {LOCATION}

{
  "id": "/subscriptions/1b365fe2-5882-4935-bd81-8027e0816b45/resourceGroups/mladsgpuaustraliaeast-test",
  "location": "australiaeast",
  "managedBy": null,
  "name": "mladsgpuaustraliaeast-test",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": null
}


In [7]:
# Setup public-ip for each vm by using `--public-ip-per-vm` parameter
!az vmss create -g {RG_NAME} -n {VMSS_NAME} --instance-count {NUM_VM} --image microsoft-dsvm:linux-data-science-vm-ubuntu:linuxdsvmubuntu:latest --vm-sku {VM_SIZE} --public-ip-per-vm --admin-username jiata --admin-password cvbpMlads2019

{- Finished ..
  "vmss": {
    "doNotRunExtensionsOnOverprovisionedVMs": false,
    "overprovision": true,
    "provisioningState": "Succeeded",
    "singlePlacementGroup": true,
    "uniqueId": "b37d6073-6744-4bda-939a-116ca8f19b4e",
    "upgradePolicy": {
      "mode": "Manual"
    },
    "virtualMachineProfile": {
      "networkProfile": {
        "networkInterfaceConfigurations": [
          {
            "name": "mlads2b46Nic",
            "properties": {
              "dnsSettings": {
                "dnsServers": []
              },
              "enableAcceleratedNetworking": false,
              "enableIPForwarding": false,
              "ipConfigurations": [
                {
                  "name": "mlads2b46IPConfig",
                  "properties": {
                    "loadBalancerBackendAddressPools": [
                      {
                        "id": "/subscriptions/1b365fe2-5882-4935-bd81-8027e0816b45/resourceGroups/mladsgpuaustraliaeast-test/providers/Microsof

In [19]:
# Check public ip addresses of the instances
!az vmss list-instance-public-ips --resource-group {RG_NAME} --name {VMSS_NAME} | grep 'ipAddress'

    "ipAddress": "104.210.81.79",
    "ipAddress": "104.210.81.80",
    "ipAddress": "104.210.81.93",
    "ipAddress": "104.210.81.109",


## Create User Accounts and Environment Setup

In [20]:
%%writefile setup_vm.sh

cd ~

# clone repo and install the conda env 
git clone https://www.github.com/microsoft/computervision 
# change permission as we copy this into each user's folder
chmod -R ugo+rwx /root/computervision

source /data/anaconda/etc/profile.d/conda.sh
conda env create -f /root/computervision/environment.yml --name cv
conda activate cv 
python -m ipykernel install --name cv --display-name "MLADS CV LAB" 

# add the 5 users to jupyterhub
echo 'c.Authenticator.whitelist = {"mlads1", "mlads2", "mlads3", "mlads4", "mlads5"}' | sudo tee -a /etc/jupyterhub/jupyterhub_config.py

# create the users to the vm 
for i in {1..5}
do
    USERNAME=mlads$i
    PASSWORD=cvmlads$i
    sudo adduser --quiet --disabled-password --gecos "" $USERNAME
    echo "$USERNAME:$PASSWORD" | sudo chpasswd
    rm -rf /data/home/$USERNAME/notebooks/*
    # copy repo
    cp -ar /root/computervision /data/home/$USERNAME/notebooks
done

# restart jupyterhub 
sudo systemctl stop jupyterhub 
sudo systemctl start jupyterhub 

exit

Writing setup_vm.sh


In [10]:
!az vmss list-instances -n {VMSS_NAME} -g {RG_NAME} --query "[].id" --output tsv | az vmss run-command invoke --command-id RunShellScript --scripts @{SCRIPT} --ids @- 

[- Finished ..
  {
    "value": [
      {
        "code": "ProvisioningState/succeeded",
        "displayStatus": "Provisioning succeeded",
        "level": "Info",
        "message": "Enable succeeded: \n[stdout]\ntebooks\"->azureml-sdk[contrib,notebooks]>=1.0.30->-r /root/computervision/tmp1dYkq3.requirements.txt (line 3)) (0.7.1)\nBuilding wheels for collected packages: nvidia-ml-py3, pycocotools, fusepy, pathspec\n  Building wheel for nvidia-ml-py3 (setup.py): started\n  Building wheel for nvidia-ml-py3 (setup.py): finished with status 'done'\n  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-cp36-none-any.whl size=19192 sha256=2034d11a4dfb093340eb35b48851591ec37eac709ad2bfa48443adf563b67227\n  Stored in directory: /root/.cache/pip/wheels/e4/1d/06/640c93f5270d67d0247f30be91f232700d19023f9e66d735c7\n  Building wheel for pycocotools (setup.py): started\n  Building wheel for pycocotools (setup.py): finished with status 'done'\n  Created wheel for pycocotools: filename=